In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
from tqdm import tqdm
import matplotlib.pyplot as plt

# Configuration
class Config:
    SR = 32000  # Sampling rate
    N_MELS = 128  # Number of Mel bands
    MAX_SEQ_LEN = 200  # Maximum sequence length for input to the model
    ROOT_FOLDER = '/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/'  # Dataset root folder
    SUBSET_SIZE = 50000  # Size of the subset to use
    OUTPUT_FOLDER = '/content/drive/MyDrive/dataset/TeamDeepwave/dataset/preprocessed/'  # Folder to save PNG images

CONFIG = Config()

# Ensure output directories exist
os.makedirs(os.path.join(CONFIG.OUTPUT_FOLDER, 'train', 'real'), exist_ok=True)
os.makedirs(os.path.join(CONFIG.OUTPUT_FOLDER, 'train', 'fake'), exist_ok=True)
os.makedirs(os.path.join(CONFIG.OUTPUT_FOLDER, 'test'), exist_ok=True)

# Function to load file paths and labels from CSV and select a subset for training data
def load_train_file_paths_and_labels(csv_path, subset_size=CONFIG.SUBSET_SIZE):
    df = pd.read_csv(csv_path)
    print("CSV Columns:", df.columns)  # Debug print to check columns
    print(df.head())  # Print first few rows to check structure
    df_subset = df.sample(n=subset_size)
    file_paths = df_subset['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x)).tolist()
    labels = df_subset['label'].tolist()
    return file_paths, labels

# Function to load file paths from CSV and select a subset for testing data
def load_test_file_paths(csv_path, subset_size=CONFIG.SUBSET_SIZE):
    df = pd.read_csv(csv_path)
    print("CSV Columns:", df.columns)  # Debug print to check columns
    print(df.head())  # Print first few rows to check structure
    df_subset = df.sample(n=subset_size)
    file_paths = df_subset['path'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x)).tolist()
    return file_paths

# Function to save Mel-spectrogram as PNG
def save_mel_spectrogram(file_path, label, output_folder, is_test=False):
    y, sr = librosa.load(file_path, sr=CONFIG.SR)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=CONFIG.N_MELS)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spectrogram_db, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-spectrogram')
    plt.tight_layout()

    filename = os.path.basename(file_path).replace('.ogg', '.png')
    if is_test:
        output_path = os.path.join(output_folder, filename)
    else:
        if label == 'fake':
            output_path = os.path.join(output_folder, 'fake', filename)
        else:
            output_path = os.path.join(output_folder, 'real', filename)

    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    plt.savefig(output_path)
    plt.close()

# Load file paths and labels for a subset of 50000 training files
train_files, train_labels = load_train_file_paths_and_labels('/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/train.csv')

# Load file paths for a subset of 50000 testing files
test_files = load_test_file_paths('/content/drive/MyDrive/dataset/TeamDeepwave/dataset/open/test.csv')

# Save Mel-spectrograms as PNG images for training data
for file_path, label in tqdm(zip(train_files, train_labels), total=len(train_files)):
    save_mel_spectrogram(file_path, label, os.path.join(CONFIG.OUTPUT_FOLDER, 'train'))

# Save Mel-spectrograms as PNG images for testing data
for file_path in tqdm(test_files, total=len(test_files)):
    save_mel_spectrogram(file_path, None, os.path.join(CONFIG.OUTPUT_FOLDER, 'test'), is_test=True)

print('Mel-spectrogram images saved successfully.')

CSV Columns: Index(['id', 'path', 'label'], dtype='object')
         id                  path label
0  RUNQPNJF  ./train/RUNQPNJF.ogg  real
1  JFAWUOGJ  ./train/JFAWUOGJ.ogg  fake
2  RDKEKEVX  ./train/RDKEKEVX.ogg  real
3  QYHJDOFK  ./train/QYHJDOFK.ogg  real
4  RSPQNHAO  ./train/RSPQNHAO.ogg  real
CSV Columns: Index(['id', 'path'], dtype='object')
           id                   path
0  TEST_00000  ./test/TEST_00000.ogg
1  TEST_00001  ./test/TEST_00001.ogg
2  TEST_00002  ./test/TEST_00002.ogg
3  TEST_00003  ./test/TEST_00003.ogg
4  TEST_00004  ./test/TEST_00004.ogg


  0%|          | 0/50000 [00:00<?, ?it/s]<ipython-input-2-7b871e4244e5>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=CONFIG.SR)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
  0%|          | 0/50000 [00:00<?, ?it/s]<ipython-input-2-7b871e4244e5>:46: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=CONFIG.SR)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 49%|████▉     | 24598/50000 [4:03:08<310:39:38, 44.03s/it]

In [ ]:
def load_file_paths_and_labels(csv_path, subset_size=200):
    df = pd.read_csv(csv_path)
    print("CSV Columns:", df.columns)  # Debug print to check columns
    print(df.head())  # Print first few rows to check structure
    df_subset = df.sample(n=subset_size, random_state=CONFIG.SEED)
    file_paths = df_subset['filepath'].apply(lambda x: os.path.join(CONFIG.ROOT_FOLDER, x)).tolist()
    labels = df_subset['class'].apply(lambda x: [1, 0] if x == 'fake' else [0, 1]).tolist()
    return file_paths, labels
